# Вебинар 5. Домашнее задание

#### 1) Прочитать статьи про BPR, WARP loss

#### 2) Сделать грид серч текущей модели, смотрите на метрику precision@5, считаем на тесте нашей функцией

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.sparse import csr_matrix, coo_matrix

from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM

import os, sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from lightfm.evaluation import precision_at_k, recall_at_k

from src.metrics import precision_at_k as custom_precision, recall_at_k
from src.utils import prefilter_items

/usr/local/lib/python3.9/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [23]:
def init_model(learning_rate=0.01, item_alpha=0.4, user_alpha=0.1):
    model = LightFM(no_components=60,
                    loss='bpr', # "logistic","bpr"
                    learning_rate=learning_rate,
                    item_alpha=item_alpha,
                    user_alpha=user_alpha,
                    random_state=42,
                    k=5,
                    n=15,
                    max_sampled=100)
    return model

def fit_model(model):
    return model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
              sample_weight=coo_matrix(user_item_matrix),
              user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
              item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
              epochs=20,
              num_threads=20,
              verbose=True)

def predict_model(model):
    predictions = model.predict(user_ids=users_ids_row,
                                item_ids=items_ids_row,
                                user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                num_threads=10)
    return predictions

def update_predict_results(predictions, df_result_for_metrics):
    data_train_filtered['score'] = predictions

    predict_result = data_train_filtered[['user_id','item_id','score']][data_train_filtered.item_id != 999999].drop_duplicates().sort_values(by=['user_id','score'], ascending=False).groupby('user_id')['item_id'].unique().reset_index()

    df_result_for_metrics = df_result_for_metrics.merge(predict_result, on='user_id', how='inner')

    return df_result_for_metrics

In [2]:
data = pd.read_csv('data/retail_train.csv')

item_features = pd.read_csv('data/product.csv')
user_features = pd.read_csv('data/hh_demographic.csv')

item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head()

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


In [5]:
item_features.head()

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [6]:
user_features.head()

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16


In [7]:
user_features['age_desc'].unique()

array(['65+', '45-54', '25-34', '35-44', '19-24', '55-64'], dtype=object)

In [8]:
user_features['marital_status_code'].unique()

array(['A', 'U', 'B'], dtype=object)

In [9]:
user_features['household_size_desc'].unique()

array(['2', '3', '4', '1', '5+'], dtype=object)

In [10]:
data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [13]:
n_items_before = data_train['item_id'].nunique()

data_train_filtered = prefilter_items(data_train, take_n_popular=5000, item_features=item_features)

n_items_after = data_train_filtered['item_id'].nunique()
print(f'items {n_items_before} ==> {n_items_after}')

items 86865 ==> 5001


In [15]:
user_item_matrix = pd.pivot_table(data_train_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', 
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) 
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head()

item_id,117847,818981,819255,819308,819400,819487,819590,819594,819840,819845,...,15926775,15926844,15926886,15972074,15972298,15972565,15972790,16100266,16729299,16729415
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

test_user_item_matrix = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float)

In [17]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [18]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head()

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
user_feat.shape

(2497, 7)

In [20]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head()

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
117847,450.0,NUTRITION,National,REFRIGERATED,SOY/RICE MILK,64 OZ
818981,194.0,GROCERY,National,COLD CEREAL,ALL FAMILY CEREAL,10.4 OZ
819255,1007.0,MEAT-PCKGD,National,BREAKFAST SAUSAGE/SANDWICHES,ROLLS - PORK,1 LB
819308,2879.0,MEAT,National,BEEF,CHOICE BEEF,
819400,584.0,GROCERY,National,DOG FOODS,DOG TREATS (SOFT TREATS),5.6 OZ


In [21]:
item_feat.shape

(5001, 6)

In [27]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())
users_ids_row = data_train_filtered['user_id'].apply(lambda x: userid_to_id[x]).values.astype(int)
items_ids_row = data_train_filtered['item_id'].apply(lambda x: itemid_to_id[x]).values.astype(int)

### Grid Search

In [28]:
def grid_search(learning_rate=None, item_alpha=None, user_alpha=None):
    if learning_rate is None:
        learning_rate = [0.01]
    if item_alpha is None:
        item_alpha = [0.4]
    if user_alpha is None:
        user_alpha = [0.1]
    df_result_for_metrics = result
    predict_results = pd.DataFrame()
    for learning_rate_ in learning_rate:
        for item_alpha_ in item_alpha:
            for user_alpha_ in user_alpha:

                model = init_model(learning_rate=learning_rate_, item_alpha=item_alpha_, user_alpha=user_alpha_)
                fit_model(model)
                predictions = predict_model(model)

                df_result_for_metrics = update_predict_results(predictions, df_result_for_metrics)

                precision = round(df_result_for_metrics.apply(lambda row: custom_precision(row['item_id'], row['actual'],k=5), axis=1).mean(), 4)

                kol_name = f'learning_rate: {learning_rate_} item_alpha: {item_alpha_} user_alpha: {user_alpha_}'
                df_result_for_metrics = df_result_for_metrics.rename(columns={'item_id': kol_name})

                predict_results = predict_results.append(pd.DataFrame([precision], columns=['precision'], index=[kol_name]))

                print(f"Precision: {precision} | {kol_name}")

    predict_results = predict_results.sort_values(by=['precision'], ascending=False)
    best = f'Лучшие параметры: {predict_results.head(1).precision.tolist()[0]}, {predict_results.head(1).index.tolist()[0]}'
    print(f'\n{best}')

    return df_result_for_metrics, predict_results, best

In [29]:
all_results, predict_results, best = grid_search(learning_rate = [0.01, 0.05],
                          item_alpha = [0.4, 0.3],
                          user_alpha = [0.1, 0.2])

Epoch: 100%|████████████████████████████████████| 20/20 [00:39<00:00,  1.96s/it]


Precision: 0.1369 | learning_rate: 0.01 item_alpha: 0.4 user_alpha: 0.1


Epoch: 100%|████████████████████████████████████| 20/20 [00:40<00:00,  2.01s/it]


Precision: 0.1423 | learning_rate: 0.01 item_alpha: 0.4 user_alpha: 0.2


Epoch: 100%|████████████████████████████████████| 20/20 [00:44<00:00,  2.21s/it]


...

Precision: 0.1333 | learning_rate: 0.05 item_alpha: 0.4 user_alpha: 0.2


Epoch: 100%|████████████████████████████████████| 20/20 [00:47<00:00,  2.39s/it]


Precision: 0.0623 | learning_rate: 0.05 item_alpha: 0.3 user_alpha: 0.1


Epoch: 100%|████████████████████████████████████| 20/20 [00:47<00:00,  2.36s/it]


Precision: 0.0622 | learning_rate: 0.05 item_alpha: 0.3 user_alpha: 0.2

Лучшие параметры: 0.1442, learning_rate: 0.01 item_alpha: 0.3 user_alpha: 0.1


In [30]:
predict_results

,precision
learning_rate: 0.01 item_alpha: 0.3 user_alpha: 0.1,0.1442
learning_rate: 0.01 item_alpha: 0.4 user_alpha: 0.2,0.1423
learning_rate: 0.01 item_alpha: 0.3 user_alpha: 0.2,0.1421
learning_rate: 0.01 item_alpha: 0.4 user_alpha: 0.1,0.1369
learning_rate: 0.05 item_alpha: 0.4 user_alpha: 0.2,0.1333
learning_rate: 0.05 item_alpha: 0.3 user_alpha: 0.1,0.0623
learning_rate: 0.05 item_alpha: 0.3 user_alpha: 0.2,0.0622
learning_rate: 0.05 item_alpha: 0.4 user_alpha: 0.1,0.0518
